In [103]:
import pandas as pd
import os
from tqdm import tqdm
import json

In [104]:
def load_dataset_as_df(file_path):
    df = pd.read_csv(file_path, delimiter=",", quotechar="|", encoding="utf16")
    return df

In [105]:
# validate all files
folder = "./output_action_level/writer_100_review_40_v5/submit"

writer_id_wrong_file_path_list = []

for root, subdir, file_list in os.walk(folder):
    for file_name in tqdm(file_list, desc="all files"):

        if file_name.endswith(".json"):
            continue
        #endif

        ## load metadata
        metadata_file_name = file_name.replace("_single_df.csv", "_metadata.json")
        metadata_file_path = os.path.join(folder, metadata_file_name)
        with open(metadata_file_path, mode="r", encoding="utf16") as fin:
            metadata_json_obj = json.load(fin)
        #endwith
        difficulty_mode = metadata_json_obj["difficulty"]


        file_path = os.path.join(root, file_name)
        df = load_dataset_as_df(file_path)
        result_1 = df[(df['novel_instance']==0) & (df['real_writer_id'] > 100)]
        result_2 = df[(df['novel_instance']==1) & (df['real_writer_id'] < 100)]
        
        # if len(result_1) > 0 or len(result_2) > 0:
        #     print(f"1, 2 --> {file_path}")



        # ######### for action_level_novelty the novelty should only come from the story dataset ########

        # Amazon writer id in total -> 310
        # KNOWN writer id: max -> 100, min -> 1
        # UNKNOWN writer id: max -> 310, min -> 101      ---> should be 0
        # SHIPPING writer id : max -> 1034, min -> 311   ---> should be 0
        # STORY writer id: max -> 1079, min -> 1035

        result_3 = df[(df['novel_instance']==1) & (101 <= df['real_writer_id']) & (df['real_writer_id'] <= 310 )]
        result_4 = df[(df['novel_instance']==1) & (311 <= df['real_writer_id']) & (df['real_writer_id'] <= 1034)]

        result_5 = df[(df['novel_instance']==1) & (1035 <= df['real_writer_id']) & (df['real_writer_id'] <= 1079)]

        if len(result_3) > 0 or len(result_4) > 0:
            writer_id_wrong_file_path_list.append(file_path)
            #print(file_path)
        #endif

        for index, row in result_5.iterrows():
            text = row["text"]
            text_len = len(text.strip().split())
            if difficulty_mode == "hard":
                try:
                    assert text_len >= 100 and text_len <= 150
                except:
                    print(f"easy: {text_len}")


            if difficulty_mode == "easy":
                try:
                    assert text_len >= 200 and text_len <= 300
                except:
                    print(f"hard: {text_len}")




print(len(writer_id_wrong_file_path_list))



all files: 100%|██████████| 432/432 [00:08<00:00, 51.61it/s]

0
